In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import datetime
import pandas as pd


class chdtv_news:
    
    def __init__(self, keyword, result = []):
        
        self.keyword = keyword
        self.result = result
    
    def get_chdtv_news(self, date):
        
        starttime = datetime.datetime.now()
        print(starttime)

        url = "https://www.chinatimes.com/search/{}?chdtv".format(self.keyword)
        #爬取特定頁數
        #url ="https://www.chinatimes.com/search/{}?page=2&chdtv".format(self.keyword)

        headers = {
        "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "cache-control": "max-age=0",
        "referer": "https://www.chinatimes.com/?chdtv",
        "sec-ch-ua-mobile": "?0",
        "sec-fetch-dest": "document",
        "sec-fetch-mode": "navigate",
        "sec-fetch-site": "same-origin",
        "sec-fetch-user": "?1",
        "upgrade-insecure-requests": "1",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36"

        }
        while True:

            res = requests.get(url,headers = headers)

            soup = BeautifulSoup(res.text, "html.parser")

            for col in soup.select("div.col"):
                article_date = col.select_one("span.date").text
                article_hour = col.select_one("span.hour").text
                
                try:
                    #try-except防止文章遺失造成程式停止
                    each_article_text= ""                           #建立空字串方便後面加入文章內容
                    article_url = col.select_one("h3 a")["href"]    #取得文章url
                    title = col.select_one("h3 a").text             #取得文章標題



                    print(article_date)
                    print(title)
                    print(article_url)


                    #設定時間區間
                    article_date_str = "".join(article_date.split("/"))
                    if int(article_date_str) < int(date):
                        break

                    else:

                        headers["referer"] = article_url
                        res2 = requests.get(article_url, headers = headers)
                        soup2 = BeautifulSoup(res2.text, "html.parser")

                        each_article = {}  #建立空字典方便接下來暫存統整每篇文章內容

                        paragraphs = soup2.select("div.article-body p")

                        for p in paragraphs:
                            each_article_text += p.text

                        print(each_article_text)
                        each_article_text = each_article_text.strip("(時報資訊)")
                        each_article_text = each_article_text.strip("(工商 )")
                        each_article_text = each_article_text.strip("(中時新聞網)")
                        each_article_text = each_article_text.strip("（袁延壽）") #去除文末的引用資訊

                        #加入時間，產業類別，股票代號，股票名稱，標題，網址，內容

                        each_article["關鍵字"] = self.keyword
                        each_article["標題"] = title
                        each_article["日期"] = article_date
                        each_article["時間"] = article_hour
                        each_article["網址"] = article_url
                        each_article["內容"] = each_article_text


                        #將每一篇文章資訊加入list
                        self.result.append(each_article)
                        
                        #以防萬一邊爬邊存 
                        df_new = pd.DataFrame.from_records(self.result)
                        df_new.to_csv("new_chdtv_covid19_news.csv", encoding = "utf-8-sig", index = False)


                        #設定隨機休息時間
                        sleep_time = random.randint(3,10)
                        print("sleep time: %s sec"%(sleep_time))
                        time.sleep(sleep_time)
                        
                except:
                    pass

            try:
                url = "https://www.chinatimes.com" + soup.select("a.page-link")[-2]['href'] + "&chdtv"
                if int(article_date_str) < int(date):
                        break

                else:
                    continue

            except:
                break
                
                
        endtime = datetime.datetime.now()
        
        return "finished"
    
    def get_content(self):
        df = pd.DataFrame.from_records(self.result)
        df.to_csv("chdtv_covid19_news.csv", encoding = "utf-8-sig", index = False)
        #columns = [關鍵字,標題,日期,時間,網址,內容] 
        
        return "OK"
    
    def append_content(self):
        df = pd.DataFrame.from_records(self.result)
        df.to_csv("chdtv_covid19_news.csv", mode='a', encoding = "utf-8-sig", index = False)
        
        return "OK"


In [ ]:
A = chdtv_news("新冠肺炎")
A.get_chdtv_news("20191201")

In [ ]:
#abc = A.get_content()

In [ ]:
b =A.append_content()